In [0]:
# Setup de Importação 
import sys
import os

# Pega o caminho do notebook atual
caminho_notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
caminho_workspace = f"/Workspace{caminho_notebook}"

# Sobe dois níveis para achar a Raiz
pasta_notebooks = os.path.dirname(caminho_workspace)
raiz_projeto = os.path.dirname(pasta_notebooks)

#. Adiciona ao Path
sys.path.append(raiz_projeto)
print(f"Raiz do projeto adicionada: {raiz_projeto}")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, trim, regexp_replace, lit, current_timestamp
from pyspark.sql.types import IntegerType, LongType
from delta.tables import *
from src.utils.dataframe_utils import *
from src.utils.delta_utils import salvar_delta_merge

In [0]:
# Definição dos Widgets
dbutils.widgets.text("camada_origem", "bronze", "1. Schema Origem")
dbutils.widgets.text("camada_destino", "silver", "2. Schema Destino")
dbutils.widgets.text("tabela", "api_ibge_6579", "3. Nome Tabela")

# Captura dos valores
schema_origem = dbutils.widgets.get("camada_origem")
schema_destino = dbutils.widgets.get("camada_destino")
nome_tabela = dbutils.widgets.get("tabela")

# Define caminhos completos
tabela_bronze = f"{schema_origem}.{nome_tabela}"
tabela_silver = f"{schema_destino}.{nome_tabela}"

print(f"Schema Origem: {schema_origem}")
print(f"Schema Destino: {schema_destino}")
print(f"Tabela: {nome_tabela}")
print(f"Tabela Bronze: {tabela_bronze}")
print(f"Tabela Silver: {tabela_silver}")

In [0]:
# Leitura da Bronze
df_bronze = spark.read.table(tabela_bronze)

In [0]:
# Lista das colunas desnecessárias
colunas_para_remover = [
    "MC",    
    "MN",
    "D2N",
    "D3C",      
]

# Aplicação da função de utils para excluir as colunas selecionadas
df_limpo = excluir_colunas(df_bronze, colunas_para_remover)

In [0]:
# Dicionário dos novos nomes das colunas
mapa_colunas = {
    "NC": "cod_nivel_territorial",
    "NN": "desc_nivel_territorial",
    "V": "populacao",              
    "D1C": "id_ibge",              
    "D1N": "nome_local",           
    "D2C": "ano",                  
    "D3N": "desc_variavel"         
}

# Aplicação da função de utils para renomear as colunas selecionadas
df_renomeado = renomear_colunas(df_limpo, mapa_colunas)

In [0]:
# Executar função em utils que analisa se há nulos nas colunas do dfs
analisar_nulos(df_renomeado)

In [0]:
# Lista de colunas para converter para Inteiros
cols_para_int = ["cod_nivel_territorial", "id_ibge", "ano"] 
# Lista de colunas para converter para Longos
cols_para_long = ["populacao"]

# Aplicando a função utils para converter os tipos
df_tipado = converter_tipo_colunas(df_renomeado, cols_para_int, IntegerType())
df_silver = converter_tipo_colunas(df_tipado, cols_para_long, LongType())

In [0]:
df_deduplicado = deduplicar_registros(
    df=df_silver,
    chaves_unicas=["id_ibge", "ano"],
    coluna_ordenacao="data_ingestao_bronze"
)

In [0]:
# Rastreabilidade
df_final = df_deduplicado.withColumn("data_processamento_silver", current_timestamp())

In [0]:
# Garante o Schema Silver
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_destino}")

In [0]:
# Chaves de Negócio para o MERGE no Delta Lake
chaves_silver = ["id_ibge", "ano"]

salvar_delta_merge(
    df=df_final,
    tabela_destino=tabela_silver,
    chaves_match=chaves_silver,
    modo_carga_inicial="overwrite",
    fazer_update=True
)